In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
import pandas as pd 
import io
import urllib.request as req 
import bs4
import numpy as np
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from google.colab import files
from transformers import BertTokenizer,BertModel,BertConfig
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import logging
logging.set_verbosity_error()

爬取爛番茄上的電影評論:
1. 取8000多則評論以及評分當訓練集(下載成csv檔)
2. 取得2022年的所有電影評論以及評分(下載成csv檔)
3. 3.5顆星以上的被視為正面評論，以下的被視為負面評論

In [ ]:
h_page=[]
h_review=[]
star_array=[]
p_array=[]
star_2022_array = []
p_2022_array = []
count = 0
def collect(firsturl):
    with req.urlopen(firsturl) as response:
        data=response.read().decode("utf-8")
    root=bs4.BeautifulSoup(data,'html.parser')
    movies = root.find_all("div",class_= 'row countdown-item')
    for movie in movies:
        global count
        print(count) 
        count+=1     
        name_page = movie.select('h2 > a')
        url_page = name_page[0].get("href")
        if url_page!=None:
            h_page.append(url_page) 
            #點進movie個別頁面
            with req.urlopen(url_page) as response:
                data=response.read().decode("utf-8")
            root=bs4.BeautifulSoup(data,"html.parser")
            reviews = root.find_all("div",class_='thumbnail-scoreboard-wrap')
            for review in reviews:
                name_review = review.select('score-board > a')
                url_review = name_review[1].get("href")
                if (url_review!=None):
                    url_review = "https://www.rottentomatoes.com"+url_review
                    h_review.append(url_review)
                    #點進評論區
                    with req.urlopen(url_review) as response:
                        data=response.read().decode("utf-8")
                    root=bs4.BeautifulSoup(data,'html.parser')
                    audiences = root.find_all("div",class_= 'audience-reviews__review-wrap')
                    for audience in audiences:
                        star=0;
                        #取得評論
                        p=audience.p.getText()
                        #取得星星數
                        s=audience.span.span.select('span')
                        for i in range(5):
                            if (s[i].get('class')[0]=='star-display__filled'):
                                star+=1;
                            if (s[i].get('class')[0]=='star-display__half'):
                                star+=0.5;
                        if (firsturl=="https://editorial.rottentomatoes.com/guide/best-movies-2022/"):
                            p_2022_array.append(p)
                            if (star>=3.5):
                                star_2022_array.append(1)
                            else :
                                star_2022_array.append(0)
                        else:
                            p_array.append(p)
                            if (star>=3.5):
                                star_array.append(1)
                            else :
                                star_array.append(0)

collect("https://editorial.rottentomatoes.com/guide/2021-best-movies/")
collect("https://editorial.rottentomatoes.com/guide/the-best-movies-of-2020/")
collect("https://editorial.rottentomatoes.com/guide/best-movies-of-2019/")
collect("https://editorial.rottentomatoes.com/guide/best-movies-of-2018/")
collect("https://editorial.rottentomatoes.com/guide/best-movies-2022/")
numpy_p = np.array(p_array)
numpy_star = np.array(star_array)
DF_tr = pd.DataFrame({'comment':p_array,
                    'goodorbad':star_array})
DF_2022 = pd.DataFrame({'comment':p_2022_array,
                        'goodorbad':star_2022_array})
DF_tr.to_csv('commenttr.csv', encoding = 'utf-8-sig') 
files.download('commenttr.csv')
DF_2022.to_csv('comment2022.csv', encoding = 'utf-8-sig') 
files.download('comment2022.csv')

上傳用來訓練模型的檔案，以及等待預測的檔案

In [3]:
uploaded = files.upload()
new_uploaded = files.upload() 
csv_df = pd.read_csv(io.StringIO(uploaded['commenttr.csv'].decode('utf-8')), index_col=[0])
csv_df_2022 = pd.read_csv(io.StringIO(new_uploaded['comment2022.csv'].decode('utf-8')), index_col=[0])

Saving commenttr.csv to commenttr.csv


Saving comment2022.csv to comment2022.csv


可觀察正評與負評中各個單詞出現的頻率

In [4]:
dirgood = {}
dirbad = {}
finallist=[]
s=0
hi =list(csv_df.itertuples(index=False, name=None))

for sentence,score in hi:
  list1 = sentence.split()
  set1 = set(list1)
  list2 = list(set1)
  for i in range(len(list2)):
    if not list2[i] in dirgood:
      dirgood[list2[i]]=0
    if not list2[i] in dirbad:
      dirbad[list2[i]]=0
    for j in range(len(list1)):
      if list2[i] == list1[j]:
        if (score==1):
          dirgood[list2[i]] +=1
        else:
          dirbad[list2[i]] +=1
  s+=1
print(dirgood)
print(dirbad)
import operator
new_dictgood = dict(sorted(dirgood.items(), key=operator.itemgetter(1),reverse=True))
new_dictbad = dict(sorted(dirbad.items(), key=operator.itemgetter(1),reverse=True))
print(new_dictgood)
print(new_dictbad)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



利用bert-base-uncased這個模型裡的tokenizer對文本做預處理，會回傳一個字典分別有下列三個鍵值:
1. input_ids: 文句裡各個token在辭典中的編碼
2. token_type_ids : 文句裡各個token在是在文本中的第幾行
3. attention_mask : token是否為mask 

In [4]:
x=list(csv_df['comment'])
y=list(csv_df['goodorbad'])
x_2022=list(csv_df_2022['comment'])
y_2022=list(csv_df_2022['goodorbad'])
x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encoding = tokenizer(x_train, truncation=True, padding=True, max_length=64)
test_encoding = tokenizer(x_test, truncation=True, padding=True, max_length=64)
predict_2022_encoding = tokenizer(x_2022, truncation=True, padding=True, max_length=64)
#print(train_encoding.keys())

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

**torch.utils.data.Dataset**

Pytorch提供兩種數據集:此為其中一種Map式的數據集，必須要重寫以下兩個函數:
1. __getitem__(self, idx) : 提供idx給dataloader使其能找到對應的data使其能找到對應的data
2. __len__(self):回傳數據集的長度

In [5]:
class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    #因為之後的DataLoader需要依照batchsize一批批的將傳輸的資料，封裝成訓練模型可用的輸入，所以需要idx使其能找到對應的資料
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item
    #回傳數據集的長度
    def __len__(self):
        return len(self.labels)

將數據集透過NewsDataset以及DataLoader將數據集封裝成訓練模型可用的輸入，其中因為2022的電影評論是我們用來predict的資料，所以不須隨機打亂資料(shuffle=False)

In [6]:
train_dataset = NewsDataset(train_encoding, y_train)
test_dataset = NewsDataset(test_encoding, y_test)
predict_2022_dataset = NewsDataset(predict_2022_encoding, y_2022)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)
predict_2022_dataloader = DataLoader(predict_2022_dataset, batch_size=16, shuffle=False)#不須打亂(shuffle=False)

利用 define_model(nn.Module) 建構BERT分類模型

In [7]:
class define_model(nn.Module):
    def __init__(self, freeze_bert=False, hidden_size=768):
        super().__init__()
        config = BertConfig.from_pretrained('bert-base-uncased')
        config.update({'output_hidden_states':True})
        self.bert = BertModel.from_pretrained("bert-base-uncased",config=config)
        self.fc = nn.Linear(hidden_size*4, 2)
        
        
        #是否要停止使用bert，讓參數不能參與更新
        #freeze_bert預設為False
        if freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask)
        hidden_states = torch.stack(outputs[2]) #將輸出的tuple以矩陣儲存
        #取得最後四層輸出並進行拼接
        concat_last_four_layers = torch.cat((hidden_states[-1],hidden_states[-2],hidden_states[-3],hidden_states[-4]), dim=-1)
        take_cls = concat_last_four_layers[:,0,:]  #取 [CLS] 這個token對應最後四層concat後的輸出            
        logits = self.fc(take_cls)
        return logits

1. 透過torch.device回傳使用的是cuda還是cpu
2. 創建model
3. 定義損失函數
4. 優化模型

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#回傳使用的是cuda還是cpu
print('已成功取得',device)
model = define_model().to(device)#創建model
criterion = nn.CrossEntropyLoss().to(device)#定義損失函數
#優化模型
#p.requires_grad會回傳參數是否被凍結，若被凍結就過濾掉
#反向傳播需要更新的參數
optim = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
total_steps = len(train_loader) * 1
scheduler = get_linear_schedule_with_warmup(optim, 
          num_warmup_steps = 0, # Default value in run_glue.py
          num_training_steps = total_steps)


已成功取得 cuda


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


訓練模型依照epoch次數執行

In [19]:
def train():
    model.train()
    total_train_loss = 0
    iter_num = 0
    total_iter = len(train_loader)
    for batch in train_loader:
        optim.zero_grad()#先將梯度歸零
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)                
        total_train_loss += loss.item()
           
        loss.backward()#反向傳播計算得到每個參數的梯度值
        nn.utils.clip_grad_norm_(model.parameters(), 1.0) #梯度的裁剪
        
        # 通過梯度下降執行一部參數更新
        optim.step()
        scheduler.step()

        iter_num += 1
        if(iter_num % 100==0):
            print("epoth: %d, iter_num: %d, loss: %.4f, %.2f%%" % (epoch, iter_num, loss.item(), iter_num/total_iter*100))
        
    print("Epoch: %d, Average training loss: %.4f"%(epoch, total_train_loss/len(train_loader)))

精度計算以及將test_dataloader做驗證，取得準確率

In [20]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
def validation():
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask) 
            loss = criterion(outputs, labels)
            logits = outputs
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()
            total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
    print("Accuracy: %.4f" % (avg_val_accuracy))
    print("Average testing loss: %.4f"%(total_eval_loss/len(test_dataloader)))
    print("-------------------------------")

開始訓練模型

In [21]:
for epoch in range(3):
    print("------------Epoch: %d ----------------" % epoch)
    train()
    validation()

------------Epoch: 0 ----------------
epoth: 0, iter_num: 100, loss: 0.0846, 25.13%
epoth: 0, iter_num: 200, loss: 0.4351, 50.25%
epoth: 0, iter_num: 300, loss: 0.2580, 75.38%
Epoch: 0, Average training loss: 0.2523
Accuracy: 0.8638
Average testing loss: 0.3305
-------------------------------
------------Epoch: 1 ----------------
epoth: 1, iter_num: 100, loss: 0.4293, 25.13%
epoth: 1, iter_num: 200, loss: 0.2481, 50.25%
epoth: 1, iter_num: 300, loss: 0.2811, 75.38%
Epoch: 1, Average training loss: 0.2518
Accuracy: 0.8638
Average testing loss: 0.3317
-------------------------------
------------Epoch: 2 ----------------
epoth: 2, iter_num: 100, loss: 0.2967, 25.13%
epoth: 2, iter_num: 200, loss: 0.0559, 50.25%
epoth: 2, iter_num: 300, loss: 0.3286, 75.38%
Epoch: 2, Average training loss: 0.2504
Accuracy: 0.8638
Average testing loss: 0.3308
-------------------------------


把輸入的文本分類成正面評論或負面評論，將資料集放入並取得預測的labels，並計算準確率

In [22]:
def get_predictions(model, dataloader, compute_or_not=False):
    predictions = None
    correct = 0
    total = 0
    stri = "strng"  
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, pred = torch.max(outputs.data, 1)
            # 用來計算訓練集的分類準確率
            if compute_or_not:
                labels = batch['labels'].to(device)
                total += labels.size(0)
                correct += (pred == labels).sum().item()
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))#依照batch size把分類後的結果整合
    if compute_or_not:#若compute_or_not設為true，及回傳分
        acc = correct / total
        return predictions, acc
    return predictions

把2022年的電影評論拿去預測，計算出正評以及複評的比率，get_predictions會回傳預測的解果，以及準確率

In [23]:
predictions = get_predictions(model, predict_2022_dataloader ,compute_or_not=True)
#print(predictions)
pre = predictions[0].tolist()
#print(pre)
count=pre.count(1)
#print(count)
print("acc:",predictions[1])
print("good rate:",float(count/len(pre)))

acc: 0.9027777777777778
good rate: 0.7361111111111112


輸入想要爬取的電影名稱:會得到10則評論，並將這10則評論放進模型裡預測，即可觀察預測的成效。

In [26]:

h_page=[]
h_review=[]
star_array=[]
p_array=[]
star_2022_array = []
p_2022_array = []
movie_name = input("please enter movie name : ")               
def collect(firsturl):
    with req.urlopen(firsturl) as response:
        data=response.read().decode("utf-8")
    root=bs4.BeautifulSoup(data,'html.parser')
    movies = root.find_all("tiles-carousel-responsive-item", slot="tile")
    r=0
    #print("hi")
    #name = movies[0].find_all("span")
    #print(name[0].getText())
    for movie in movies: 

        if movie.find_all("span")[0].getText()==movie_name and r==0:
            #print("here")
            #print(movie.a.get("href"))
            r=1
            url_page = "https://www.rottentomatoes.com"+movie.a.get("href")
            if url_page!=None:
                h_page.append(url_page) 
                #點進movie個別頁面
                with req.urlopen(url_page) as response:
                    data=response.read().decode("utf-8")
                root=bs4.BeautifulSoup(data,"html.parser")
                reviews = root.find_all("div",class_='thumbnail-scoreboard-wrap')
                for review in reviews:
                    name_review = review.select('score-board > a')
                    url_review = name_review[1].get("href")
                    if (url_review!=None):
                        url_review = "https://www.rottentomatoes.com"+url_review
                        h_review.append(url_review)
                        with req.urlopen(url_review) as response:
                            data=response.read().decode("utf-8")
                        root=bs4.BeautifulSoup(data,'html.parser')
                        audiences = root.find_all("div",class_= 'audience-reviews__review-wrap')
                        for audience in audiences:
                            star=0;
                            s=audience.span.span.select('span')
                            #if (audience.span.get('class')[0]=='audience-reviews__score'):
                            for i in range(5):
                                if (s[i].get('class')[0]=='star-display__filled'):
                                    star+=1;
                                if (s[i].get('class')[0]=='star-display__half'):
                                    star+=0.5;
                            p=audience.p.getText()
                            if (firsturl=="https://editorial.rottentomatoes.com/guide/best-movies-2022/"):
                                p_2022_array.append(p)
                                if (star>=3.5):
                                    star_2022_array.append(1)
                                else :
                                    star_2022_array.append(0)
                            else:
                                p_array.append(p)
                                if (star>=3.5):
                                    star_array.append(1)
                                else :
                                    star_array.append(0)
      

collect("https://www.rottentomatoes.com")
numpy_p = np.array(p_array)
numpy_star = np.array(star_array)
DF = pd.DataFrame({'comment':p_array,
                    'goodorbad':star_array})
print(DF)
input_x=list(DF['comment'])
input_y=list(DF['goodorbad'])
input_encoding = tokenizer(input_x, truncation=True, padding=True, max_length=64)
input_dataset = NewsDataset(input_encoding, input_y)
input_dataloader = DataLoader(input_dataset, batch_size=16, shuffle=False)
predictions = get_predictions(model, input_dataloader ,compute_or_not=True)
print(predictions)

please enter movie name : M3GAN
                                             comment  goodorbad
0  Great way to start the year with a thriller an...          1
1  One of the best doll movie to hit the big scre...          1
2  really good never slow really thought megan wa...          1
3  Movie was great!! Had a good laugh with my hus...          1
4           Robocop meets chucky meets the exorcist!          1
5  Good movie, had some funny parts to it. Defini...          0
6                     Slooooooow start decent ending          0
7  The ending could've used a little more detaili...          1
8        Very cheesy movie, was hoping for much more          0
9  Not worth 80 to watch this movie could of been...          0
(tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0], device='cuda:0'), 0.7)
